In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xckw8dgn
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-xckw8dgn
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [2]:
!pip install gradio

In [3]:
import torch
import clip
from PIL import Image
import os
import itertools
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using: ", device)

model, preprocess = clip.load("ViT-B/32", device=device)

#dataset
dataset_folder = '/content/data'

images = []
count = 0
for root, dirs, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith(('jpg','jpeg')):
            images.append(  root  + '/'+ file)



Using:  cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
def imagetoImage(input_image, useCosineSimilarity):
    #input image embedding
    input_image = Image.open(input_image)
    preprocess_input_image = preprocess(input_image).unsqueeze(0).to(device)
    input_image_embedding = model.encode_image(preprocess_input_image)

    result = {}
    if useCosineSimilarity:
        cosine_similarity = torch.nn.CosineSimilarity(dim=0)

        for img in images:
            with torch.no_grad():
                image_preprocess = preprocess(Image.open(img)).unsqueeze(0).to(device)
                image_features = model.encode_image(image_preprocess)
                similarity_score = cosine_similarity(image_features[0],input_image_embedding[0]).item()
                similarity_score = (similarity_score+1)/2 # makes the similarity score between 0 and 1
                result[img]=similarity_score
                print(similarity_score)

    else:
        euclidean_distance = torch.nn.PairwiseDistance()

        for img in images:
            with torch.no_grad():
                image_preprocess = preprocess(Image.open(img)).unsqueeze(0).to(device)
                image_features = model.encode_image(image_preprocess)
                distance = euclidean_distance(image_features[0], input_image_embedding[0]).item()
                similarity_score = 1 / (1 + distance)
                result[img] = similarity_score



    results = dict(sorted(result.items(), key = lambda x: x[1], reverse=True))
    print(results)
    top_3 = dict(itertools.islice(results.items(), 3))
    resulting_image = []
    print(top_3)

    for key in top_3:
        resulting_image.append(key)

    return input_image, resulting_image[0], resulting_image[1], resulting_image[2]

In [6]:

def image_to_image(image, value):
    flag = False
    if value[0] != "Cosine Similarity":
      flag = True
    return imagetoImage(image, flag)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import torch
import clip
from PIL import Image
import os
import itertools
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using: ", device)

model, preprocess = clip.load("ViT-B/32", device=device)

Using:  cuda


In [9]:
def buildCocoData():
  images = []
  embeddings = []
  dataset_folder = '/content/drive/MyDrive/coco_val2017/val2017'
  count = 0
  for root, dirs, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith(('jpg','jpeg')):
            x = root  + '/'+ file
            images.append(x)
            tensor = model.encode_image(preprocess(Image.open(x)).unsqueeze(0).to(device))
            embeddings.append(tensor)
            count = count + 1
            if count == 300:
              break
            print(count)

  return images, embeddings

In [10]:
images, embeddings = buildCocoData()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
def textToImage(input_text, useCosineSimilarity):
  text_token = clip.tokenize(input_text).to(device)
  text_embedding = model.encode_text(text_token).to(device)
  result = []
  if useCosineSimilarity:
        cosine_similarity = torch.nn.CosineSimilarity(dim=0)

        for index, emb in enumerate(embeddings):
            with torch.no_grad():
                similarity_score = cosine_similarity(emb[0], text_embedding[0]).item()
                similarity_score = (similarity_score+1)/2 # makes the similarity score between 0 and 1
                result.append((index, images[index], similarity_score))

  else:
        euclidean_distance = torch.nn.PairwiseDistance()

        for index, emb in enumerate(embeddings):
            with torch.no_grad():
                similarity_score = euclidean_distance(emb[0], text_embedding[0]).item()
                similarity_score = 1/(similarity_score+1) # makes the similarity score between 0 and 1
                result.append((index, images[index], similarity_score))

  results = sorted(result, key=lambda tup: tup[2])
  top_3 = results[len(results) - 3 :]
  print(results)
  return top_3[2][1], top_3[1][1], top_3[0][1]

In [12]:
def text_to_image(image, value):
    flag = True
    if value[0] != "Cosine Similarity":
      flag = False
    return textToImage(image, flag)

In [13]:
textToImage("A computer sitting on a desk", True)

[(30, '/content/drive/MyDrive/coco_val2017/val2017/000000269316.jpg', 0.5285186767578125), (154, '/content/drive/MyDrive/coco_val2017/val2017/000000247917.jpg', 0.530181884765625), (244, '/content/drive/MyDrive/coco_val2017/val2017/000000183648.jpg', 0.540496826171875), (209, '/content/drive/MyDrive/coco_val2017/val2017/000000288685.jpg', 0.541015625), (96, '/content/drive/MyDrive/coco_val2017/val2017/000000414170.jpg', 0.54107666015625), (89, '/content/drive/MyDrive/coco_val2017/val2017/000000478721.jpg', 0.545166015625), (27, '/content/drive/MyDrive/coco_val2017/val2017/000000285894.jpg', 0.545196533203125), (261, '/content/drive/MyDrive/coco_val2017/val2017/000000058705.jpg', 0.545501708984375), (49, '/content/drive/MyDrive/coco_val2017/val2017/000000064718.jpg', 0.54583740234375), (285, '/content/drive/MyDrive/coco_val2017/val2017/000000556158.jpg', 0.545989990234375), (103, '/content/drive/MyDrive/coco_val2017/val2017/000000527695.jpg', 0.54620361328125), (67, '/content/drive/MyDr

('/content/drive/MyDrive/coco_val2017/val2017/000000038576.jpg',
 '/content/drive/MyDrive/coco_val2017/val2017/000000066635.jpg',
 '/content/drive/MyDrive/coco_val2017/val2017/000000570782.jpg')

In [14]:
!pip install open_clip_torch

In [21]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import open_clip

def getModel(modelName, pretrainedName):
  model, _, preprocess = open_clip.create_model_and_transforms(modelName, pretrained=pretrainedName)
  tokenizer = open_clip.get_tokenizer(modelName)
  return model, tokenizer, preprocess

def getTextEmbeddings(prompts, classes, model, tokenizer, device):
  textEmbeddings = None
  model_outputs = []
  with torch.no_grad():
    for promptType in prompts:
      promptList = []
      for classType in classes:
        promptEngineered = promptType + classType
        promptList.append(promptEngineered)
      tokenizePrompt = tokenizer(promptList).to(device)
      textEmbeddings = model.encode_text(tokenizePrompt)
      textEmbeddings = textEmbeddings / torch.linalg.vector_norm(textEmbeddings, dim = 1, keepdim = True)
      model_outputs.append(textEmbeddings)
  return model_outputs

def zeroShotClassification(images, model, allTextEmbeddings, device):
  imageEmbedding = None
  with torch.no_grad():
      imageEmbedding = model.encode_image(images.to(device)).to(device)
      imageEmbedding = imageEmbedding / torch.linalg.vector_norm(imageEmbedding, dim = 1, keepdim = True)
  print("image embedded")
  model_outputs = []
  with torch.no_grad():
    for textEmbeddings in allTextEmbeddings:
      model_outputs.append(imageEmbedding.mm(textEmbeddings.T))

  return torch.stack(model_outputs)

def getLabel(output):
  imageCount = torch.zeros((output.shape[1], output.shape[2]))
  for prompt in output:
    rowIndex = 0
    for imageRow in prompt:
      softmaxOutput = torch.softmax(imageRow, dim = -1)
      index = torch.argmax(softmaxOutput)
      imageCount[rowIndex][index] += 1
      rowIndex += 1
  ans = []
  for image in imageCount:
    index = torch.argmax(image)
    ans.append(index)
  return torch.tensor(ans).reshape(-1)

def zero_shot(imageName, prompts, classes):
  prompts = prompts.split("\n")
  classes = classes.split("\n")
  model, tokenizer, preprocess = getModel('ViT-B-32', 'laion2b_s34b_b79k')
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  model.to(device).eval()
  embeddings = getTextEmbeddings(prompts, classes, model, tokenizer, device)
  data = preprocess(Image.open(imageName)).unsqueeze(0)
  data.to(device)
  print("Reached")
  ans = zeroShotClassification(data, model, embeddings, device)
  print("Reached2")
  output_labels = getLabel(ans).to(device)
  return imageName, classes[output_labels[0]]

In [23]:
import numpy as np
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("CLIP Semantic Segmentation")
    with gr.Tab("Text to Text Search"):
        with gr.Row():
          text_input = gr.Textbox(label="input text")
          text_checkbox = gr.CheckboxGroup(["Cosine Similarity", "Eucledian Distance"], label="Similarity Option")
          image_text_output = [gr.Image(label="result 1"), gr.Image(label="result 2"), gr.Image(label="result 3")]
        text_button = gr.Button("Submit")
    with gr.Tab("Image to Image Search"):
        with gr.Row():
            image_input = gr.Textbox(label = "input image file location")
            image_checkbox = gr.CheckboxGroup(["Cosine Similarity", "Eucledian Distance"], label="Similarity Option")
            image_output = [gr.Image(label = "input image"), gr.Image(label="result 1"), gr.Image(label="result 2"), gr.Image(label="result 3")]
        image_button = gr.Button("Submit")
    with gr.Tab("Zero Shot Classification"):
        with gr.Row():
            image_zero_input = gr.Textbox(label = "input image filename")
            prompt_input = gr.Textbox(label="prompts", lines=20)
            classes_input = gr.Textbox(label="classes", lines=20)
            text_output = [gr.Image(label = "input image"), gr.Textbox(label="output class")]
        zero_button = gr.Button("Submit")

    text_button.click(text_to_image, inputs=[text_input, text_checkbox], outputs=image_text_output)
    image_button.click(image_to_image, inputs=[image_input, image_checkbox], outputs=image_output)
    zero_button.click(zero_shot, inputs=[image_zero_input, prompt_input, classes_input], outputs=text_output)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3abd7aba51a77cb10c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
